<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/create_dataset/sensor_data_feature_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This kernel is introducing feature store for Indoor Location & Navigation competition, because the dataset of this competition is much comlicated and raw data. So it is useful for easy to retrieve data.

This script is not finalized. It will be updated for retrieve much more information.

In [4]:
import json
import re
import gc
import pickle
import itertools
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime as dt
from pathlib import Path
from tqdm import tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # ut(10 digit) -> date

# pandas settings -----------------------------------------
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.5f}'.format

# Graph drawing -------------------------------------------
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc
from matplotlib_venn import venn2, venn2_circles
from matplotlib import animation as ani
from IPython.display import Image
from pylab import imread

plt.rcParams["patch.force_edgecolor"] = True
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

%matplotlib inline
%config InlineBackend.figure_format='retina'

# ML -------------------------------------------
from sklearn.preprocessing import LabelEncoder

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def unpickle(filename):
    with open(filename, 'rb') as fo:
        p = pickle.load(fo)
    return p

def to_pickle(filename, obj):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, -1)



class FeatureStore():
    
    # necessayr to re-check
    floor_convert = {'1F' :  0, '2F' : 1, '3F' : 2, '4F' : 3, '5F' : 4, 
                     '6F' : 5, '7F' : 6, '8F' : 7, '9F' : 8,
                     'B'  : -1, 'B1' : -1, 'B2' : -2, 'B3' : -3, 
                     'BF' : -1, 'BM' : -1, 
                     'F1' : 0, 'F2' : 1, 'F3' : 2, 'F4' : 3, 'F5' : 4, 
                     'F6' : 5, 'F7' : 6, 'F8' : 7, 'F9' : 8, 'F10': 9,
                     'L1' : 0, 'L2' : 1, 'L3' : 2, 'L4' : 3, 'L5' : 4, 
                     'L6' : 5, 'L7' : 6, 'L8' : 7, 'L9' : 8, 'L10': 9, 
                     'L11': 10,
                     'G'  : 0, 'LG1': 0, 'LG2': 1, 'LM' : 0, 'M'  : 0, 
                     'P1' : 0, 'P2' : 1,}
    
    df_types = ['accelerometer',
                'accelerometer_uncalibrated',
                'beacon',
                'gyroscope',
                'gyroscope_uncalibrated',
                'magnetic_field',
                'magnetic_field_uncalibrated',
                'rotation_vector',
                'waypoint',
                'wifi']
    
    # https://github.com/location-competition/indoor-location-competition-20
    df_type_cols = {'accelerometer': ["timestamp", "x", "y", "z", "accuracy"],
                'accelerometer_uncalibrated': ["timestamp", "x", "y", "z", 
                                               "x2", "y2", "z2", "accuracy" ],
                'beacon': ["timestamp", "uuid", "major_id", "minor_id", "tx_power", 
                           "rssi", "distance", "mac_addr", "timestamp2"],
                'gyroscope': ["timestamp", "x", "y", "z", "accuracy"],
                'gyroscope_uncalibrated': ["timestamp", "x", "y", "z", 
                                           "x2", "y2", "z2", "accuracy" ],
                'magnetic_field': ["timestamp", "x", "y", "z", "accuracy"],
                'magnetic_field_uncalibrated': ["timestamp", "x", "y", "z", 
                                                "x2", "y2", "z2", "accuracy" ],
                'rotation_vector': ["timestamp", "x", "y", "z", "accuracy"],
                'waypoint': ["timestamp", "x", "y"],
                'wifi': ["timestamp", "ssid", "bssid","rssi","frequency",
                         "last_seen_timestamp",]}

    dtype_dict = {}
    dtype_dict["accelerometer"] = {"timestamp":int, "x":float, "y":float, "z":float, 
                                   "accuracy":int}
    dtype_dict["accelerometer_uncalibrated"] = {"timestamp":int, "x":float, "y":float, 
                                                "z":float, "x2":float, "y2":float, 
                                                "z2":float, "accuracy":int}
    dtype_dict["beacon"] = {"timestamp":int, "uuid":str, "major_id":str, 
                            "minor_id":str, "tx_power":int,  "rssi":int, 
                            "distance":float, "mac_addr":str, "timestamp2":int}
    dtype_dict["gyroscope"] = {"timestamp":int, "x":float, "y":float, "z":float, 
                               "accuracy":int}
    dtype_dict["gyroscope_uncalibrated"] = {"timestamp":int, "x":float, "y":float, 
                                            "z":float, "x2":float, "y2":float, 
                                            "z2":float, "accuracy":int}
    dtype_dict["magnetic_field"] = {"timestamp":int, "x":float, "y":float, 
                                    "z":float, "accuracy":int}
    dtype_dict["magnetic_field_uncalibrated"] = {"timestamp":int, "x":float, 
                                                 "y":float, "z":float, "x2":float, 
                                                 "y2":float, "z2":float, "accuracy":int}
    dtype_dict["rotation_vector"] = {"timestamp":int, "x":float, "y":float, 
                                     "z":float, "accuracy":int}
    dtype_dict["waypoint"] = {"timestamp":int, "x":float, "y":float, "z":float}
    dtype_dict["wifi"] = {"timestamp":int, "ssid":str, "bssid":str,
                          "rssi":int,"frequency":int, "last_seen_timestamp":int}

    def __init__(self, site_id, floor, path_id, 
                 input_path="/content/drive/MyDrive/fixed_train",
                 save_path="../mid"):
        self.site_id = site_id.strip()
        self.floor = floor.strip()
        self.n_floor = self.floor_convert[self.floor]
        self.path_id = path_id.strip()
        
        self.input_path = input_path
        assert Path(input_path).exists(), f"input_path do not exist: {input_path}"
        
        self.save_path = save_path
        Path(save_path).mkdir(parents=True, exist_ok=True)
        
        self.site_info = SiteInfo(site_id=self.site_id, floor=self.floor, input_path=self.input_path)
        
    def _flatten(self, l):
        return list(itertools.chain.from_iterable(l))
    
    def multi_line_spliter(self, s):
        matches = re.finditer("TYPE_", s)
        matches_positions = [match.start() for match in matches]
        split_idx = [0] + [matches_positions[i]-14 for i in range(1, len(matches_positions))] + [len(s)]
        return [s[split_idx[i]:split_idx[i+1]] for i in range(len(split_idx)-1)]
    
    def load_df(self, ):
        path = str(Path(self.input_path)/f"train/{self.site_id}/{self.floor}/{self.path_id}.txt")
        with open(path) as f:
            data = f.readlines()
        
        modified_data = []
        for s in data:
            if s.count("TYPE_")>1:
                lines = self.multi_line_spliter(s)
                modified_data.extend(lines)
            else:
                modified_data.append(s)
        del data
        self.meta_info_len = len([d for d in modified_data if d[0]=="#"])
        self.meta_info_df = pd.DataFrame([m.replace("\n", "").split(":") 
                                          for m in self._flatten([d.split("\t") 
                                                                  for d in modified_data if d[0]=="#"]) if m!="#"])

        data_df = pd.DataFrame([d.replace("\n", "").split("\t") for d in modified_data if d[0]!="#"])
        for dt in self.df_types:
            # select data type
            df_s = data_df[data_df[1]==f"TYPE_{dt.upper()}"]
            if len(df_s)==0:
                setattr(self, dt, pd.DataFrame(columns=self.df_type_cols[dt]))
            else:
                # remove empty cols
                na_info = df_s.isna().sum(axis=0) == len(df_s)
                df_s = df_s[[i for i in na_info[na_info==False].index if i!=1]].reset_index(drop=True)
                
                if len(df_s.columns)!=len(self.df_type_cols[dt]):
                    df_s.columns = self.df_type_cols[dt][:len(df_s.columns)]
                else:
                    df_s.columns = self.df_type_cols[dt]
            
                # set dtype          
                for c in df_s.columns:
                    df_s[c] = df_s[c].astype(self.dtype_dict[dt][c])
                                     
                # set DataFrame to attr
                setattr(self, dt, df_s)
    
    def get_site_info(self, keep_raw=False):
        self.site_info.get_site_info(keep_raw=keep_raw)
            
    def load_all_data(self, keep_raw=False):     
        self.load_df()
        
    def __getitem__(self, item):
        if item in self.df_types:
            return getattr(self, item)
        else:
            return None
    
    def save(self, ):
        # to be implemented
        pass
    
    
class SiteInfo():
    def __init__(self, site_id, floor, input_path="/content/drive/MyDrive/fixed_train"):
        self.site_id = site_id
        self.floor = floor
        self.input_path = input_path
        assert Path(input_path).exists(), f"input_path do not exist: {input_path}"
        
    def get_site_info(self, keep_raw=False):
        floor_info_path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/floor_info.json"
        with open(floor_info_path, "r") as f:
            self.floor_info = json.loads(f.read())
            self.site_height = self.floor_info["map_info"]["height"]
            self.site_width = self.floor_info["map_info"]["width"]
            if not keep_raw:
                del self.floor_info
            
        geojson_map_path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/geojson_map.json"
        with open(geojson_map_path, "r") as f:
            self.geojson_map = json.loads(f.read())
            self.map_type = self.geojson_map["type"]
            self.features = self.geojson_map["features"]
            
            self.floor_coordinates = self.features[0]["geometry"]["coordinates"]
            self.store_coordinates = [self.features[i]["geometry"]["coordinates"] 
                                          for i in range(1, len(self.features))]
                
            if not keep_raw:
                del self.geojson_map
    
    def show_site_image(self):
        path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/floor_image.png"
        plt.imshow(imread(path), extent=[0, self.site_width, 0, self.site_height])

    def draw_polygon(self, size=8, only_floor=False):

        fig = plt.figure()
        ax = plt.subplot(111)
            
        xmax, xmin, ymax, ymin = self._draw(self.floor_coordinates, ax, calc_minmax=True)
        if not only_floor:
            self._draw(self.store_coordinates, ax, fill=True)
        plt.legend([])
        
        xrange = xmax - xmin
        yrange = ymax - ymin
        ratio = yrange / xrange
        
        self.x_size = size
        self.y_size = size*ratio

        fig.set_figwidth(size)
        fig.set_figheight(size*ratio)
        # plt.show()
        return ax
        
    def _draw(self, coordinates, ax, fill=False, calc_minmax=False):
        xmax, ymax = -np.inf, -np.inf
        xmin, ymin = np.inf, np.inf
        for i in range(len(coordinates)):
            ndim = np.ndim(coordinates[i])
            if ndim==2:
                corrd_df = pd.DataFrame(coordinates[i])
                if fill:
                    ax.fill(corrd_df[0], corrd_df[1], alpha=0.7)
                else:
                    corrd_df.plot.line(x=0, y=1, style="-", ax=ax)
                        
                if calc_minmax:
                    xmax = max(xmax, corrd_df[0].max())
                    xmin = min(xmin, corrd_df[0].min())

                    ymax = max(ymax, corrd_df[1].max())
                    ymin = min(ymin, corrd_df[1].min())
            elif ndim==3:
                for j in range(len(coordinates[i])):
                    corrd_df = pd.DataFrame(coordinates[i][j])
                    if fill:
                        ax.fill(corrd_df[0], corrd_df[1], alpha=0.6)
                    else:
                        corrd_df.plot.line(x=0, y=1, style="-", ax=ax)
                        
                    if calc_minmax:
                        xmax = max(xmax, corrd_df[0].max())
                        xmin = min(xmin, corrd_df[0].min())

                        ymax = max(ymax, corrd_df[1].max())
                        ymin = min(ymin, corrd_df[1].min())
            else:
                assert False, f"ndim of coordinates should be 2 or 3: {ndim}"
        if calc_minmax:
            return xmax, xmin, ymax, ymin
        else:
            return None
        

# prepare meta information

In [ ]:
# site_meta_data
site_meta_data = pd.DataFrame([[p.split("/")[-2], p.split("/")[-1]] for p in glob("../input/indoor-location-navigation/metadata/**/*")])
site_meta_data.columns = ["site_id", "floor"]
site_meta_data.head()

,site_id,floor
0,5cd56c0ce2acfd2d33b6ab27,B1
1,5cd56c0ce2acfd2d33b6ab27,F3
2,5cd56c0ce2acfd2d33b6ab27,F2
3,5cdbc652853bc856e89a8694,B1
4,5cdbc652853bc856e89a8694,F1


In [7]:
# train_meta_data
train_meta = glob("/content/drive/MyDrive/use_data/train/*/*/*")
train_meta_org = pd.DataFrame(train_meta)
train_meta = train_meta_org[0].str.split("/", expand=True)[[6, 7, 8]]
train_meta.columns = ["site_id", "floor", "path_id"]
train_meta["path_id"] = train_meta["path_id"].str.replace(".txt", "")
train_meta["path"] = train_meta_org[0]
train_meta.head()

,site_id,floor,path_id,path
0,5a0546857ecc773753327266,B1,5e1573141506f2000638fc31,/content/drive/MyDrive/use_data/train/5a0546857ecc773753327266/B1/5e1573141506f2000638fc31.txt
1,5a0546857ecc773753327266,B1,5e158ef11506f2000638fd1b,/content/drive/MyDrive/use_data/train/5a0546857ecc773753327266/B1/5e158ef11506f2000638fd1b.txt
2,5a0546857ecc773753327266,B1,5e1580c51506f2000638fc6a,/content/drive/MyDrive/use_data/train/5a0546857ecc773753327266/B1/5e1580c51506f2000638fc6a.txt
3,5a0546857ecc773753327266,B1,5e158f1f1506f2000638fd3b,/content/drive/MyDrive/use_data/train/5a0546857ecc773753327266/B1/5e158f1f1506f2000638fd3b.txt
4,5a0546857ecc773753327266,B1,5e15730e1506f2000638fc2b,/content/drive/MyDrive/use_data/train/5a0546857ecc773753327266/B1/5e15730e1506f2000638fc2b.txt


In [10]:
train_meta[train_meta['path_id'] == '5d79bdfc8c61220006a3466a']

,site_id,floor,path_id,path
504,5a0546857ecc773753327266,F4,5d79bdfc8c61220006a3466a,/content/drive/MyDrive/use_data/train/5a0546857ecc773753327266/F4/5d79bdfc8c61220006a3466a.txt


# train data

In [13]:
a = pd.read_csv('/content/drive/MyDrive/rotation_vector/5a0546857ecc773753327266_rotation_vector_train.csv')
a.head()

,Unnamed: 0,timestamp,x,y,z,accuracy,path_id
0,0,1578463582250,0.05566,0.17295,0.87189,3.00000,5e1573141506f2000638fc31
1,1,1578463582270,0.05574,0.17334,0.87182,3.00000,5e1573141506f2000638fc31
2,2,1578463582290,0.05570,0.17245,0.87204,3.00000,5e1573141506f2000638fc31
3,3,1578463582310,0.05566,0.17147,0.87249,3.00000,5e1573141506f2000638fc31
4,4,1578463582330,0.05590,0.17056,0.87290,3.00000,5e1573141506f2000638fc31


In [9]:
ssubm = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

In [12]:
import shutil

In [ ]:
import multiprocessing
from multiprocessing import Pool

num_cores = multiprocessing.cpu_count()

In [11]:
for building in used_buildings:
    head_index = train_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = train_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = train_meta.iloc[n]
        feature = FeatureStore(site_id=t.site_id, floor=t.floor, path_id=t.path_id)
        feature.load_all_data()
        df = feature['waypoint']
        df['path_id'] = t.path_id
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_waypoint_train.csv")
    shutil.move(f'/content/{building}_waypoint_train.csv', '/content/drive/MyDrive/waypoint')

In [14]:
for building in used_buildings:
    head_index = train_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = train_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = train_meta.iloc[n]
        feature = FeatureStore(site_id=t.site_id, floor=t.floor, path_id=t.path_id)
        feature.load_all_data()
        df = feature['accelerometer']
        df['path_id'] = t.path_id
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_accelerometer_train.csv")
    print(building)
    shutil.move(f'/content/{building}_accelerometer_train.csv', '/content/drive/MyDrive/accelerometer')


5a0546857ecc773753327266
5c3c44b80379370013e0fd2b
5d27075f03f801723c2e360f
5d27096c03f801723c31e5e0
5d27097f03f801723c320d97
5d27099f03f801723c32511d
5d2709a003f801723c3251bf
5d2709b303f801723c327472
5d2709bb03f801723c32852c
5d2709c303f801723c3299ee
5d2709d403f801723c32bd39
5d2709e003f801723c32d896
5da138274db8ce0c98bbd3d2
5da1382d4db8ce0c98bbe92e
5da138314db8ce0c98bbf3a0
5da138364db8ce0c98bc00f1
5da1383b4db8ce0c98bc11ab
5da138754db8ce0c98bca82f
5da138764db8ce0c98bcaa46
5da1389e4db8ce0c98bd0547
5da138b74db8ce0c98bd4774
5da958dd46f8266d0737457b
5dbc1d84c1eb61796cf7c010
5dc8cea7659e181adb076a3f


In [ ]:
for building in used_buildings:
    head_index = train_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = train_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = train_meta.iloc[n]
        feature = FeatureStore(site_id=t.site_id, floor=t.floor, path_id=t.path_id)
        feature.load_all_data()
        df = feature['accelerometer_uncalibrated']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_accelerometer_uncalibrated_train.csv")
    shutil.move(f'/content/{building}_accelerometer_uncalibrated_train.csv', '/content/drive/MyDrive/accelerometer_uncalibrated')

In [ ]:
for building in used_buildings:
    head_index = train_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = train_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = train_meta.iloc[n]
        feature = FeatureStore(site_id=t.site_id, floor=t.floor, path_id=t.path_id)
        feature.load_all_data()
        df = feature['beacon']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_beacon_train.csv")
    shutil.move(f'/content/{building}_beacon_train.csv', '/content/drive/MyDrive/beacon')

In [ ]:
for building in used_buildings:
    head_index = train_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = train_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = train_meta.iloc[n]
        feature = FeatureStore(site_id=t.site_id, floor=t.floor, path_id=t.path_id)
        feature.load_all_data()
        df = feature['gyroscope']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_gyroscope_train.csv")
    print(building)
    shutil.move(f'/content/{building}_gyroscope_train.csv', '/content/drive/MyDrive/gyroscope')

In [ ]:
for building in used_buildings:
    head_index = train_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = train_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = train_meta.iloc[n]
        feature = FeatureStore(site_id=t.site_id, floor=t.floor, path_id=t.path_id)
        feature.load_all_data()
        df = feature['gyroscope_uncalibrated']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_gyroscope_uncalibrated_train.csv")
    shutil.move(f'/content/{building}_gyroscope_uncalibrated_train.csv', '/content/drive/MyDrive/gyroscope_uncalibrated')

In [ ]:
for building in used_buildings:
    head_index = train_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = train_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = train_meta.iloc[n]
        feature = FeatureStore(site_id=t.site_id, floor=t.floor, path_id=t.path_id)
        feature.load_all_data()
        df = feature['magnetic_field']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_magnetic_field_train.csv")
    print(building)
    shutil.move(f'/content/{building}_magnetic_field_train.csv', '/content/drive/MyDrive/magnetic_field')

In [ ]:
for building in used_buildings:
    head_index = train_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = train_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = train_meta.iloc[n]
        feature = FeatureStore(site_id=t.site_id, floor=t.floor, path_id=t.path_id)
        feature.load_all_data()
        df = feature['magnetic_field_uncalibrated']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_magnetic_field_uncalibrated_train.csv")
    shutil.move(f'/content/{building}_magnetic_field_uncalibrated_train.csv', '/content/drive/MyDrive/magnetic_field_uncalibrated')

In [ ]:
for building in used_buildings:
    head_index = train_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = train_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = train_meta.iloc[n]
        feature = FeatureStore(site_id=t.site_id, floor=t.floor, path_id=t.path_id)
        feature.load_all_data()
        df = feature['rotation_vector']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_rotation_vector_train.csv")
    print(building)
    shutil.move(f'/content/{building}_rotation_vector_train.csv', '/content/drive/MyDrive/rotation_vector')

# test data



In [ ]:
class FeatureStore_Test():
    
    # necessayr to re-check
    floor_convert = {'1F' :  0, '2F' : 1, '3F' : 2, '4F' : 3, '5F' : 4, 
                     '6F' : 5, '7F' : 6, '8F' : 7, '9F' : 8,
                     'B'  : -1, 'B1' : -1, 'B2' : -2, 'B3' : -3, 
                     'BF' : -1, 'BM' : -1, 
                     'F1' : 0, 'F2' : 1, 'F3' : 2, 'F4' : 3, 'F5' : 4, 
                     'F6' : 5, 'F7' : 6, 'F8' : 7, 'F9' : 8, 'F10': 9,
                     'L1' : 0, 'L2' : 1, 'L3' : 2, 'L4' : 3, 'L5' : 4, 
                     'L6' : 5, 'L7' : 6, 'L8' : 7, 'L9' : 8, 'L10': 9, 
                     'L11': 10,
                     'G'  : 0, 'LG1': 0, 'LG2': 1, 'LM' : 0, 'M'  : 0, 
                     'P1' : 0, 'P2' : 1,}
    
    df_types = ['accelerometer',
                'accelerometer_uncalibrated',
                'beacon',
                'gyroscope',
                'gyroscope_uncalibrated',
                'magnetic_field',
                'magnetic_field_uncalibrated',
                'rotation_vector',
                'waypoint',
                'wifi']
    
    # https://github.com/location-competition/indoor-location-competition-20
    df_type_cols = {'accelerometer': ["timestamp", "x", "y", "z", "accuracy"],
                'accelerometer_uncalibrated': ["timestamp", "x", "y", "z", 
                                               "x2", "y2", "z2", "accuracy" ],
                'beacon': ["timestamp", "uuid", "major_id", "minor_id", "tx_power", 
                           "rssi", "distance", "mac_addr", "timestamp2"],
                'gyroscope': ["timestamp", "x", "y", "z", "accuracy"],
                'gyroscope_uncalibrated': ["timestamp", "x", "y", "z", 
                                           "x2", "y2", "z2", "accuracy" ],
                'magnetic_field': ["timestamp", "x", "y", "z", "accuracy"],
                'magnetic_field_uncalibrated': ["timestamp", "x", "y", "z", 
                                                "x2", "y2", "z2", "accuracy" ],
                'rotation_vector': ["timestamp", "x", "y", "z", "accuracy"],
                'waypoint': ["timestamp", "x", "y"],
                'wifi': ["timestamp", "ssid", "bssid","rssi","frequency",
                         "last_seen_timestamp",]}

    dtype_dict = {}
    dtype_dict["accelerometer"] = {"timestamp":int, "x":float, "y":float, "z":float, 
                                   "accuracy":int}
    dtype_dict["accelerometer_uncalibrated"] = {"timestamp":int, "x":float, "y":float, 
                                                "z":float, "x2":float, "y2":float, 
                                                "z2":float, "accuracy":int}
    dtype_dict["beacon"] = {"timestamp":int, "uuid":str, "major_id":str, 
                            "minor_id":str, "tx_power":int,  "rssi":int, 
                            "distance":float, "mac_addr":str, "timestamp2":int}
    dtype_dict["gyroscope"] = {"timestamp":int, "x":float, "y":float, "z":float, 
                               "accuracy":int}
    dtype_dict["gyroscope_uncalibrated"] = {"timestamp":int, "x":float, "y":float, 
                                            "z":float, "x2":float, "y2":float, 
                                            "z2":float, "accuracy":int}
    dtype_dict["magnetic_field"] = {"timestamp":int, "x":float, "y":float, 
                                    "z":float, "accuracy":int}
    dtype_dict["magnetic_field_uncalibrated"] = {"timestamp":int, "x":float, 
                                                 "y":float, "z":float, "x2":float, 
                                                 "y2":float, "z2":float, "accuracy":int}
    dtype_dict["rotation_vector"] = {"timestamp":int, "x":float, "y":float, 
                                     "z":float, "accuracy":int}
    dtype_dict["waypoint"] = {"timestamp":int, "x":float, "y":float, "z":float}
    dtype_dict["wifi"] = {"timestamp":int, "ssid":str, "bssid":str,
                          "rssi":int,"frequency":int, "last_seen_timestamp":int}

    def __init__(self, site_id, path_id, 
                 input_path="/content/drive/MyDrive/data_24",
                 save_path="../mid"):
        self.site_id = site_id.strip()
        self.path_id = path_id.strip()
        
        self.input_path = input_path
        assert Path(input_path).exists(), f"input_path do not exist: {input_path}"
        
        self.save_path = save_path
        Path(save_path).mkdir(parents=True, exist_ok=True)
                
    def _flatten(self, l):
        return list(itertools.chain.from_iterable(l))
    
    def multi_line_spliter(self, s):
        matches = re.finditer("TYPE_", s)
        matches_positions = [match.start() for match in matches]
        split_idx = [0] + [matches_positions[i]-14 for i in range(1, len(matches_positions))] + [len(s)]
        return [s[split_idx[i]:split_idx[i+1]] for i in range(len(split_idx)-1)]
    
    def load_df(self, ):
        path = str(Path(self.input_path)/f"test/{self.site_id}/{self.path_id}.txt")
        with open(path) as f:
            data = f.readlines()
        
        modified_data = []
        for s in data:
            if s.count("TYPE_")>1:
                lines = self.multi_line_spliter(s)
                modified_data.extend(lines)
            else:
                modified_data.append(s)
        del data
        self.meta_info_len = len([d for d in modified_data if d[0]=="#"])
        self.meta_info_df = pd.DataFrame([m.replace("\n", "").split(":") 
                                          for m in self._flatten([d.split("\t") 
                                                                  for d in modified_data if d[0]=="#"]) if m!="#"])

        data_df = pd.DataFrame([d.replace("\n", "").split("\t") for d in modified_data if d[0]!="#"])
        for dt in self.df_types:
            # select data type
            df_s = data_df[data_df[1]==f"TYPE_{dt.upper()}"]
            if len(df_s)==0:
                setattr(self, dt, pd.DataFrame(columns=self.df_type_cols[dt]))
            else:
                # remove empty cols
                na_info = df_s.isna().sum(axis=0) == len(df_s)
                df_s = df_s[[i for i in na_info[na_info==False].index if i!=1]].reset_index(drop=True)
                
                if len(df_s.columns)!=len(self.df_type_cols[dt]):
                    df_s.columns = self.df_type_cols[dt][:len(df_s.columns)]
                else:
                    df_s.columns = self.df_type_cols[dt]
            
                # set dtype          
                for c in df_s.columns:
                    df_s[c] = df_s[c].astype(self.dtype_dict[dt][c])
                                     
                # set DataFrame to attr
                setattr(self, dt, df_s)

            
    def load_all_data(self, keep_raw=False):     
        self.load_df()
        
    def __getitem__(self, item):
        if item in self.df_types:
            return getattr(self, item)
        else:
            return None
    
    def save(self, ):
        # to be implemented
        pass

In [ ]:
# test_meta_data
test_meta = glob("/content/drive/MyDrive/data_24/test/*/*")
test_meta_org = pd.DataFrame(test_meta)
test_meta = pd.DataFrame(test_meta_org[0].str.split("/", expand=True)[[6, 7]])
test_meta.columns = ["site_id", "path_id"]
test_meta["path_id"] = test_meta["path_id"].str.replace(".txt", "")
test_meta["path"] = test_meta_org[0]
test_meta.head()

In [7]:
test_meta[test_meta['path_id'] == 'bfcb651b80df271b79d344dd']

,site_id,path_id,path
36,5d27075f03f801723c2e360f,bfcb651b80df271b79d344dd,/content/drive/MyDrive/data_24/test/5d27075f03f801723c2e360f/bfcb651b80df271b79d344dd.txt


In [10]:
for building in used_buildings:
    head_index = test_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = test_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = test_meta.iloc[n]
        feature = FeatureStore_Test(site_id=t.site_id, path_id=t.path_id)
        feature.load_all_data()
        df = feature['waypoint']
        df['path_id'] = t.path_id
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_waypoint_test.csv")
    print(building)
    #shutil.move(f'/content/{building}_waypoint_test.csv', '/content/drive/MyDrive/waypoint')

KeyboardInterrupt: ignored

In [18]:
a = pd.read_csv('/content/5d27075f03f801723c2e360f_accelerometer_test.csv')
a[a['path_id'] == '9cc4412ff73ec37e30be9d9f']

,Unnamed: 0,timestamp,x,y,z,accuracy,path_id
16068,0,122,-0.92923,-0.11536,10.00024,2,9cc4412ff73ec37e30be9d9f
16069,1,142,-0.89572,-0.18120,10.16305,2,9cc4412ff73ec37e30be9d9f
16070,2,162,-0.88853,-0.13809,10.02359,2,9cc4412ff73ec37e30be9d9f
16071,3,182,-0.91786,-0.10757,9.48250,2,9cc4412ff73ec37e30be9d9f
16072,4,202,-1.01782,-0.11356,9.33347,2,9cc4412ff73ec37e30be9d9f
16073,5,221,-1.09982,-0.12852,9.97211,2,9cc4412ff73ec37e30be9d9f
16074,6,241,-1.08127,-0.20154,10.37973,2,9cc4412ff73ec37e30be9d9f
16075,7,261,-1.11838,-0.22189,9.73628,2,9cc4412ff73ec37e30be9d9f
16076,8,281,-1.22311,-0.22728,9.45676,2,9cc4412ff73ec37e30be9d9f
16077,9,301,-1.31650,-0.17819,9.74765,2,9cc4412ff73ec37e30be9d9f


In [ ]:
for building in used_buildings:
    head_index = test_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = test_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = test_meta.iloc[n]
        feature = FeatureStore_Test(site_id=t.site_id, path_id=t.path_id)
        feature.load_all_data()
        df = feature['accelerometer']
        df['path_id'] = t.path_id
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_accelerometer_test.csv")
    print(building)
    #shutil.move(f'/content/{building}_accelerometer_test.csv', '/content/drive/MyDrive/accelerometer/test')

In [ ]:
for building in used_buildings:
    head_index = test_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = test_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = test_meta.iloc[n]
        feature = FeatureStore_Test(site_id=t.site_id, path_id=t.path_id)
        feature.load_all_data()
        df = feature['accelerometer_uncalibrated']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_accelerometer_uncalibrated_test.csv")
    shutil.move(f'/content/{building}_accelerometer_uncalibrated_test.csv', '/content/drive/MyDrive/accelerometer_uncalibrated/test')

In [ ]:
for building in used_buildings:
    head_index = test_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = test_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = test_meta.iloc[n]
        feature = FeatureStore_Test(site_id=t.site_id, path_id=t.path_id)
        feature.load_all_data()
        df = feature['beacon']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_beacon_test.csv")
    shutil.move(f'/content/{building}_beacon_test.csv', '/content/drive/MyDrive/beacon/test')

In [ ]:
for building in used_buildings:
    head_index = test_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = test_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = test_meta.iloc[n]
        feature = FeatureStore_Test(site_id=t.site_id, path_id=t.path_id)
        feature.load_all_data()
        df = feature['gyroscope']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_gyroscope_test.csv")
    print(building)
    shutil.move(f'/content/{building}_gyroscope_test.csv', '/content/drive/MyDrive/gyroscope/test')

In [ ]:
for building in used_buildings:
    head_index = test_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = test_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = test_meta.iloc[n]
        feature = FeatureStore_Test(site_id=t.site_id, path_id=t.path_id)
        feature.load_all_data()
        df = feature['gyroscope_uncalibrated']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_gyroscope_uncalibrated_test.csv")
    shutil.move(f'/content/{building}_gyroscope_uncalibrated_test.csv', '/content/drive/MyDrive/gyroscope_uncalibrated/test')

In [ ]:
for building in used_buildings:
    head_index = test_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = test_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = test_meta.iloc[n]
        feature = FeatureStore_Test(site_id=t.site_id, path_id=t.path_id)
        feature.load_all_data()
        df = feature['magnetic_field']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_magnetic_field_test.csv")
    print(building)
    shutil.move(f'/content/{building}_magnetic_field_test.csv', '/content/drive/MyDrive/magnetic_field/test')

In [ ]:
for building in used_buildings:
    head_index = test_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = test_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = test_meta.iloc[n]
        feature = FeatureStore_Test(site_id=t.site_id, path_id=t.path_id)
        feature.load_all_data()
        df = feature['magnetic_field_uncalibrated']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_magnetic_field_uncalibrated_test.csv")
    shutil.move(f'/content/{building}_magnetic_field_uncalibrated_test.csv', '/content/drive/MyDrive/magnetic_field_uncalibrated/test')

In [ ]:
for building in used_buildings:
    head_index = test_meta.groupby('site_id').get_group(building).head(1).index[0]
    tail_index = test_meta.groupby('site_id').get_group(building).tail(1).index[0]
    dfs = []
    for n in range(head_index,tail_index+1):
        t = test_meta.iloc[n]
        feature = FeatureStore_Test(site_id=t.site_id, path_id=t.path_id)
        feature.load_all_data()
        df = feature['rotation_vector']
        dfs.append(df)
    building_df = pd.concat(dfs)
    building_df.to_csv(building+"_rotation_vector_test.csv")
    print(building)
    shutil.move(f'/content/{building}_rotation_vector_test.csv', '/content/drive/MyDrive/rotation_vector/test')